# Concatenate tweets

In this notebook, we will consolidate all the tweet files into a single compressed pickle file for further analysis. We have three main sets of data that we need to store: 

1. Data from January 2021.
2. Data from October 2021.
3. Data from April 28 to June 30.

Each of these samples corresponds to a specific moment relevant for our analysis. The October data is used for analyzing our community during election periods, specifically the regional elections in Colombia that took place in October 2019. The data from January 2021 represents the period three months before the "Paro Nacional," allowing us to track our community before the social outbreak. Finally, we have the data from the time of the "Paro Nacional," which will be the focal point of our analysis.

In [10]:
os.listdir("../../../../../home/lgomezt")

['.bash_history',
 '.Xauthority',
 '.python_history',
 '.bashrc',
 '.config',
 '.wget-hsts',
 '.conda',
 '.cache',
 '.ipython',
 '.vscode-server',
 '.sudo_as_admin_successful',
 '.local',
 '.gitconfig']

In [5]:
import os
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm

In [2]:
path = r"/mnt/disk2/Data/"

## Regional elections: October 2019

In [ ]:
# We create an empty aux list that will store the tweets.
tweets_aux = []
files_oct = glob(os.path.join(path, 'users_oct_19/*.csv'))

for file in tqdm(files_oct):
    tweets_aux.append(pd.read_csv(file))

# Finally, the tweet dataframe is established and tweets_aux is deleted.  
tweets = pd.concat(tweets_aux)
del tweets_aux
tweets = tweets.sort_values('ID').reset_index(drop = True)

# Store results
tweets.to_pickle(os.path.join(path, "Tweets_DataFrames/tweets_oct19.gzip"), compression = "gzip")

## Before Paro Nacional: January 2021

We identify two users with their file corrupted: Usuario_82383620 and Usuario_2526574133

In [ ]:
# We create an empty aux list that will store the tweets.
tweets_aux = []
files_jan = glob(os.path.join(path, "RawData/users_jan/*.csv"))

for file in tqdm(files_jan):
    tweets_aux.append(pd.read_csv(file))

# Finally, the tweet dataframe is established and tweets_aux is deleted.  
tweets_jan = pd.concat(tweets_aux)
del tweets_aux
tweets_jan = tweets_jan.sort_values('ID').reset_index(drop = True)

# We check the DataFrame
print('Shape: ',tweets_jan.shape)
tweets_jan.head()

# Store results
# run sudo chmod 777 /mnt/disk2/Data/Tweets_DataFrames in bash if it is needed
tweets_jan.to_pickle(os.path.join(path, "Tweets_DataFrames/tweets_jan21.gzip"), compression = "gzip")

## Paro Nacional: April 28 - June 30 2021

In [3]:
files_v1 = glob(os.path.join(path, 'RawData/Usuarios_V1/*.csv'))
len(files_v1)

37324

In [46]:
df_list = []
users_information = []

# cols = ['ID', 'Author ID', 'Author Name', 'Date', 'Text', 'Replies', 'Retweets', 'Favorites', 'Quotes', 'is Retweet?',
#            'Reply To User Name', 'Mentions', 'Referenced Tweet', 'Reference Type', 'Referenced Tweet Author ID']

problems = []

def unique_to_string(x):
    unique_values = x.unique()
    return ', '.join(map(str, unique_values))

# Counter and variable for keeping track of file names
count = 0
n = 0
for file in tqdm(files_v1):
    try:
        # df = pd.read_csv(file, usecols = cols)
        df = pd.read_csv(file)
        # Fix some datatypes
        df[['Author Followers', 'Author Following', 'Author Tweets']] = df[['Author Followers', 'Author Following', 'Author Tweets']].applymap(lambda x: pd.to_numeric(x, errors = 'coerce'))
        df_list.append(df)
        count += len(df)

        # Save user information
        user_information = df.groupby(['Author ID', 'Author Name']).agg({
                'Author Location': unique_to_string,
                'Author Description': unique_to_string,
                'Author Followers': lambda x: np.nanmean(x),
                'Author Following': lambda x: np.nanmean(x),
                'Author Tweets': lambda x: np.nanmax(x),
                'Author Verified': unique_to_string})
        users_information.append(user_information)
        
        # If we reach or exceed 10 million rows, save the file and reset
        if count >= 10_000_000:
            n += 1
            concat_df = pd.concat(df_list)
            output_filename = f"tweets_paro_{n}.gzip"
            concat_df.to_pickle(os.path.join(path, f"Tweets_DataFrames/{output_filename}"), compression='gzip')
            
            # Reset counter and list
            count = 0
            df_list = []
            
    except (ValueError, KeyError) as e:
        problems.append(file)

# Save any remaining data after the loop
if df_list:
    n += 1
    concat_df = pd.concat(df_list)
    output_filename = f"tweets_paro_{n}.gzip"
    # If necessary, run "sudo chmod 777 Data/Tweets_DataFrames" in bash
    concat_df.to_pickle(os.path.join(path, f"Tweets_DataFrames/{output_filename}"), compression = 'gzip')

del df_list, concat_df

concat_users_information = pd.concat(users_information)
concat_users_information = concat_users_information.groupby(['Author ID', 'Author Name']) \
    .agg({'Author Location': unique_to_string,
            'Author Description': unique_to_string,
            'Author Followers': lambda x: np.nanmean(x),
            'Author Following': lambda x: np.nanmean(x),
            'Author Tweets': lambda x: np.nanmax(x),
            'Author Verified': unique_to_string})
concat_users_information.to_pickle(os.path.join(path, "Tweets_DataFrames/users_information.gzip"), 
                                   compression = 'gzip')

  2%|▏         | 681/37324 [00:23<22:36, 27.01it/s]/tmp/ipykernel_1123518/2326954817.py:29: RuntimeWarning: Mean of empty slice
  'Author Followers': lambda x: np.nanmean(x),
/tmp/ipykernel_1123518/2326954817.py:30: RuntimeWarning: Mean of empty slice
  'Author Following': lambda x: np.nanmean(x),
/tmp/ipykernel_1123518/2326954817.py:31: RuntimeWarning: All-NaN axis encountered
  'Author Tweets': lambda x: np.nanmax(x),
  4%|▍         | 1406/37324 [00:48<15:30, 38.59it/s]/tmp/ipykernel_1123518/2326954817.py:29: RuntimeWarning: Mean of empty slice
  'Author Followers': lambda x: np.nanmean(x),
/tmp/ipykernel_1123518/2326954817.py:31: RuntimeWarning: All-NaN axis encountered
  'Author Tweets': lambda x: np.nanmax(x),
  4%|▍         | 1421/37324 [00:48<18:52, 31.72it/s]/tmp/ipykernel_1123518/2326954817.py:29: RuntimeWarning: Mean of empty slice
  'Author Followers': lambda x: np.nanmean(x),
/tmp/ipykernel_1123518/2326954817.py:30: RuntimeWarning: Mean of empty slice
  'Author Following': 

In [47]:
# We should correct this
concat_users_information

Author Location  \
Author ID    Author Name                                
0.000000e+00 0               True, False, True, False   
1.000000e+00 0                                  False   
2.000000e+00 0                                  False   
3.000000e+00 0                            False, True   
             1                                  False   
...                                               ...   
1.389722e+18 Neoplasticista                  Colombia   
1.389737e+18 JC13177979        Bogotá, D.C., Colombia   
1.389741e+18 JhonatanVRojo         Medellín, Colombia   
1.389769e+18 VaneLen18                       Colombia   
1.389784e+18 kars0518                            COL.   

                                                            Author Description  \
Author ID    Author Name                                                         
0.000000e+00 0                      nan, equidad_mujer, mariumega, PattyRosi24   
1.000000e+00 0                                              nan, equidad_mujer   
2.000000e+00 0               nan, estoacaquees, thearchipielago, equidad_mu...   
3.000000e+00 0                           MinTransporteCo, nan, Supertransporte   
             1                                                             nan   
...                                                                        ...   
1.389722e+18 Neoplasticista                      Arquitecto. Contra Corriente.   
1.389737e+18 JC13177979      Aunque nadie ha podido regresar y hacer un nue...   
1.389741e+18 JhonatanVRojo                 El mundo es más que blanco & negro.   
1.389769e+18 VaneLen18                                                     nan   
1.389784e+18 kars0518                                                      nan   

                             Author Followers  Author Following  \
Author ID    Author Name                                          
0.000000e+00 0                            NaN      1.384355e+18   
1.000000e+00 0                            NaN      1.398276e+18   
2.000000e+00 0                            NaN      1.396578e+18   
3.000000e+00 0                            NaN      1.396701e+18   
             1                            NaN               NaN   
...                                       ...               ...   
1.389722e+18 Neoplasticista              91.0      4.980000e+02   
1.389737e+18 JC13177979                  94.0      1.780000e+02   
1.389741e+18 JhonatanVRojo              103.0      4.270000e+02   
1.389769e+18 VaneLen18                    8.0      9.300000e+01   
1.389784e+18 kars0518                    53.0      8.700000e+01   

                             Author Tweets  \
Author ID    Author Name                     
0.000000e+00 0                         NaN   
1.000000e+00 0                         NaN   
2.000000e+00 0                         NaN   
3.000000e+00 0                         NaN   
             1                         NaN   
...                                    ...   
1.389722e+18 Neoplasticista         3534.0   
1.389737e+18 JC13177979             7083.0   
1.389741e+18 JhonatanVRojo          1257.0   
1.389769e+18 VaneLen18              1179.0   
1.389784e+18 kars0518                768.0   

                                                               Author Verified  
Author ID    Author Name                                                        
0.000000e+00 0               nan, https://pbs.twimg.com/media/E2C38-kXEAIkE...  
1.000000e+00 0               nan, https://pbs.twimg.com/media/E19UAtWXsAYjz...  
2.000000e+00 0               https://pbs.twimg.com/ext_tw_video_thumb/14011...  
3.000000e+00 0               https://pbs.twimg.com/media/E0ysfTuWUAUo8by.jp...  
             1                 https://pbs.twimg.com/media/E1dlhtaXEAYTwol.jpg  
...                                                                        ...  
1.389722e+18 Neoplasticista                                              False  
1.389737e+18 JC13177979      

### Tweets Lite
We create a reduced version of the Paro data frame. This will have the same amount of rows but we will only store four columns: 'Author ID', 'Date', 'Reference Type', 'Referenced Tweet Author ID'.

In [ ]:
# Get just the columns that we need for the Graph construction
cols = ['Author ID', 'Date', 'Reference Type', 'Referenced Tweet Author ID']
tweets_lite = tweets[cols].reset_index(drop = True)
# Store results
# run sudo chmod 777 Data/Tweets_DataFrames in bash if it is needed
tweets_lite.to_pickle(os.path.join(path, "Tweets_DataFrames/tweets_lite.gzip"), compression = "gzip")

## Outoputs

The output of this Notebook are stored "/mnt/disk2/Data/Tweets_DataFrames" and are listed below:

- **tweets_jan21.gzip**: Dataframe for the Tweets for our users during January of 2021. 3 Months before the Paro
- **tweets_oct19.gzip**: Dataframe for the Tweets for our users during October of 2019. Regional elections Period
- **tweets_paro_i.gzip**: 5 dataframes for the tweets of our users between April 28 to June 30 of 2021
- **tweets_lite.pkl**: Lite version of **tweets_Usuarios_V1.gzip** that contains just the colmns needed for the graph construction. Which is Author ID, Reference Type, Date and Retweet Author